##Práctica 4. Similitud de palabras y oraciones
Ultima modificación: 10/06/2024

In [5]:
#Cargar librerias
import re 
import nltk
import os
import spacy
import numpy as np
import scipy as sp
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import transformers
import torch
from nltk.probability import FreqDist



In [6]:
#Cargar los documentos.
with open('libros/The Adventures of Sherlock Holmes.txt' ,'r', encoding = 'utf-8') as archivo:
    libro1 = archivo.read()

with open('libros/The Last Man.txt' ,'r', encoding = 'utf-8') as archivo:
    libro2 = archivo.read()

with open('libros/Frankenstein.txt' ,'r', encoding = 'utf-8') as archivo:
    libro3 = archivo.read()

with open('libros/The War of the Worlds.txt' ,'r', encoding = 'utf-8') as archivo:
    libro4 = archivo.read()

with open('libros/THE SECRET ADVERSARY.txt' ,'r', encoding = 'utf-8') as archivo:
    libro5 = archivo.read()

with open('libros/The Mysterious Affair at Styles.txt' ,'r', encoding = 'utf-8') as archivo:
    libro6 = archivo.read()

with open('libros/MOBY-DICK;.txt' ,'r', encoding = 'utf-8') as archivo:
    libro7 = archivo.read()

with open('libros/Alice’s Adventures in Wonderland.txt', 'r', encoding = 'utf-8') as archivo:
    libro8 = archivo.read()

with open('libros/DRACULA.txt', 'r', encoding= 'utf-8') as archivo:
    libro9 = archivo.read()

with open('libros/The Phantom of the Opera.txt', 'r', encoding= 'utf-8') as archivo:
    libro10 =  archivo.read()





In [7]:
#Remover indiceS
libro1 = re.sub(r'Contents.*?(?=I.\n)', '', libro1, flags=re.DOTALL)
libro2 = re.sub(r'Contents.*?(?=I visited)', '', libro2, flags=re.DOTALL)
libro3 = re.sub(r'CONTENTS.*?(?=England._\n)', '', libro3, flags=re.DOTALL)
libro4 = re.sub(r'But who shall dwell.*?(?=No one would)', '', libro4, flags=re.DOTALL)
libro5 = re.sub(r'CONTENTS.*?AND AFTER', '', libro5, flags=re.DOTALL)
libro6 = re.sub(r'Contents.*?(?=EXPLAINS\n)', '', libro6, flags=re.DOTALL)
libro7 = re.sub(r'CONTENTS.*?(?=Call me Ishmael)', '', libro7, flags=re.DOTALL)
libro8 = re.sub(r'THE MILLENNIUM FULCRUM EDITION 3\.0.*?(?=Alice was)', '', libro8, flags= re.DOTALL)
libro9 = re.sub(r'Contents.*?(?=DRACULA\n)', '', libro9, flags=re.DOTALL)
libro10 = re.sub(r'Contents.*?(?=\{plus a "bonus chapter" called "THE PARIS OPERA HOUSE"\}\n)', '', libro10, flags=re.DOTALL)


#Extacción del primer capitulo y creación del corpus.

In [ ]:
#Extraer el primer capitulo de cada libro
patron = re.compile(r"I.\n(.*?)II\.", re.DOTALL)
resultado1 = patron.search(libro1)

patron = re.compile(r"I am the native of.*?II", re.DOTALL)
resultado2 = patron.search(libro2)

patron = re.compile(r"Chapter 1.*?Chapter 2", re.DOTALL)
resultado3 = patron.search(libro3)

patron = re.compile(r"No one would.*?II", re.DOTALL)
resultado4 = patron.search(libro4)

patron = re.compile(r"CHAPTER I.*?CHAPTER II", re.DOTALL)
resultado5 = patron.search(libro5)

patron = re.compile(r"CHAPTER I.*?CHAPTER II", re.DOTALL)
resultado6 = patron.search(libro6)

patron = re.compile(r"Call me Ishmael(.*?)CHAPTER 2", re.DOTALL)
resultado7 = patron.search(libro7)

patron = re.compile(r"Alice was.*?CHAPTER II", re.DOTALL)
resultado8 = patron.search(libro8)

patron = re.compile(r"CHAPTER I.*?CHAPTER II", re.DOTALL)
resultado9 = patron.search(libro9)

patron = re.compile(r"Chapter I.*?Chapter II", re.DOTALL)
resultado10 = patron.search(libro10)

# Crear una carpeta para almacenar los capítulos
os.makedirs('libros', exist_ok=True)

# Función para guardar el primer capítulo en un archivo
def guardar_capitulo(nombre_archivo, contenido):
    if contenido:
        with open(f'libros/{nombre_archivo}', 'w', encoding='utf-8') as f:
            f.write(contenido)

# Guardar los primeros capítulos en archivos separados
guardar_capitulo('capitulo1libro1.txt', resultado1.group(1))
guardar_capitulo('capitulo1libro2.txt', resultado2.group(0))
guardar_capitulo('capitulo1libro3.txt', resultado3.group(0))
guardar_capitulo('capitulo1libro4.txt', resultado4.group(0))
guardar_capitulo('capitulo1libro5.txt', resultado5.group(0))
guardar_capitulo('capitulo1libro6.txt', resultado6.group(0))
guardar_capitulo('capitulo1libro7.txt', resultado7.group(0))
guardar_capitulo('capitulo1libro8.txt', resultado8.group(0))
guardar_capitulo('capitulo1libro9.txt', resultado9.group(0))
guardar_capitulo('capitulo1libro10.txt', resultado10.group(0))

# Crear un nuevo corpus con los primeros capítulos
corpus_root = 'libros'
capitulos = [f'capitulo1libro{i}.txt' for i in range(1, 11)]
new_corpus = PlaintextCorpusReader(corpus_root, capitulos)

# Verificar el contenido del corpus
print(new_corpus.fileids())

['capitulo1libro1.txt', 'capitulo1libro2.txt', 'capitulo1libro3.txt', 'capitulo1libro4.txt', 'capitulo1libro5.txt', 'capitulo1libro6.txt', 'capitulo1libro7.txt', 'capitulo1libro8.txt', 'capitulo1libro9.txt', 'capitulo1libro10.txt']


#Segmentación en oraciones y tokenizacion de acuerdo a su etiquetado gramatical.

In [9]:
etiquetas_por_archivo = {}
# Procesar cada archivo de capítulo por separado
for archivo in capitulos:
    # Leer el contenido del archivo
    texto = new_corpus.raw(archivo)
    
    # Dividir el texto en oraciones
    oraciones = nltk.sent_tokenize(texto)
    
    etiquetas_oraciones = []
    
    # Tokenizar y etiquetar gramaticalmente cada oración
    for oracion in oraciones:
        tokens = nltk.word_tokenize(oracion)
        etiquetas = nltk.pos_tag(tokens)
        etiquetas_oraciones.append(etiquetas)
    
    # Guardar las etiquetas gramaticales por archivo en el diccionario
    etiquetas_por_archivo[archivo] = etiquetas_oraciones


#SIMILITUD SEMÁNTICA DE PALABRAS Y DOCUMENTOS


##Verbos mas frecuentes y terminos similares.

In [10]:
verbos_mas_comunes = []

#Identificar el verbo más común en cada capítulo
for archivo, etiquetas_oraciones in etiquetas_por_archivo.items():
    print(f'Archivo: {archivo}')
    verbos = []
    for oracion in etiquetas_oraciones:
        for palabra, etiqueta in oracion:
            if etiqueta.startswith('VB') and palabra not in stopwords.words('english') and palabra not in ['’', "'s"]:
                verbos.append(palabra)
    fdist = FreqDist(verbos)
    verbo_mas_comun = fdist.most_common(1)
    
    # Guardar el verbo más común en la lista
    if verbo_mas_comun:
        verbos_mas_comunes.append(verbo_mas_comun[0][0])
    else:
        verbos_mas_comunes.append(None)
    
    # Imprimir el verbo más común encontrado en el archivo
    if verbo_mas_comun:
        print(f'Verbo más común encontrado: {verbo_mas_comun[0]}')
    else:
        print('No se encontraron verbos en este archivo.')
    
    print()
    

# Función para obtener los términos más similares utilizando path_similarity
def obtener_terminos_similares(verbo):
    # Obtener los synsets del verbo en WordNet
    synsets = wn.synsets(verbo, pos=wn.VERB)
    
    terminos_similares = []
    
    # Iterar sobre cada synset del verbo
    for synset in synsets:
        # Calcular la similitud de camino con otros synsets
        similitudes = []
        for otro_synset in wn.all_synsets(pos=wn.VERB):
            similitud = synset.path_similarity(otro_synset)
            if similitud is not None:
                similitudes.append((otro_synset, similitud))
        
        # Ordenar por similitud y obtener los términos similares
        similitudes.sort(key=lambda x: x[1], reverse=True)
        terminos_similares.extend((syn.name().split('.')[0] for syn, _ in similitudes[:5]))
    
    return terminos_similares

# Función para obtener los términos más similares utilizando path_similarity
def obtener_terminos_similares2(verbo):
    # Obtener los synsets del verbo en WordNet
    synsets = wn.synsets(verbo, pos=wn.VERB)
    
    terminos_similares = []
    
    # Iterar sobre cada synset del sustantivo
    for synset in synsets:
        # Calcular la similitud de camino con otros synsets
        similitudes = []
        for otro_synset in wn.all_synsets(pos=wn.VERB):
            similitud = synset.wup_similarity(otro_synset)
            if similitud is not None:
                similitudes.append((otro_synset, similitud))
        
        # Ordenar por similitud y obtener los términos similares
        similitudes.sort(key=lambda x: x[1], reverse=True)
        terminos_similares.extend((syn.name().split('.')[0] for syn, _ in similitudes[:5]))
    
    return terminos_similares


# Crear una lista para almacenar los términos similares para cada verbo más común
terminos_similares = []

# Iterar sobre cada verbo más común utilizando path_similarity
for verbo in verbos_mas_comunes:
    terminos = obtener_terminos_similares(verbo)
    terminos_similares.append(terminos)

# Imprimir los términos similares para cada verbo más común
print(f'Terminos similares aplicando path_similarity\n')
for i, terminos in enumerate(terminos_similares):
    print(f'Términos similares para el verbo más común del archivo {capitulos[i]}:')
    print(terminos[0:5])
    print()

terminos_similares2 = []
# Iterar sobre cada verbo más común utilizando wup_similarity
for verbo in verbos_mas_comunes:
    terminos = obtener_terminos_similares2(verbo)
    terminos_similares2.append(terminos)

print('\n\n Terminos similares utilizando wup_similarity\n')
for i, terminos in enumerate(terminos_similares2):
    #print(f'Términos similares para el verbo más común del archivo {capitulos[i]}:')
    print(terminos[0:5])
    print()

Archivo: capitulo1libro1.txt
Verbo más común encontrado: ('said', 21)

Archivo: capitulo1libro2.txt
Verbo más común encontrado: ('came', 5)

Archivo: capitulo1libro3.txt
Verbo más común encontrado: ('became', 7)

Archivo: capitulo1libro4.txt
Verbo más común encontrado: ('remember', 6)

Archivo: capitulo1libro5.txt
Verbo más común encontrado: ('know', 12)

Archivo: capitulo1libro6.txt
Verbo más común encontrado: ('said', 14)

Archivo: capitulo1libro7.txt
Verbo más común encontrado: ('go', 10)

Archivo: capitulo1libro8.txt
Verbo más común encontrado: ('see', 10)

Archivo: capitulo1libro9.txt
Verbo más común encontrado: ('seemed', 24)

Archivo: capitulo1libro10.txt
Verbo más común encontrado: ('said', 14)

Terminos similares aplicando path_similarity

Términos similares para el verbo más común del archivo capitulo1libro1.txt:
['state', 'present', 'misstate', 'answer', 'precede']

Términos similares para el verbo más común del archivo capitulo1libro2.txt:
['come', 'emanate', 'address', 'tr

#Sustantivos mas frecuentes y terminos similares

In [11]:
all_synsets_nouns = list(wn.all_synsets(pos=wn.NOUN))[:4000]  # Limitar a los primeros 4000

# Función para obtener los términos más similares utilizando path_similarity
def obtener_terminos_similares(sustantivo):
    synsets = wn.synsets(sustantivo, pos=wn.NOUN)
    terminos_similares = []

    for synset in synsets:
        similarities = []
        for otro_synset in all_synsets_nouns:
            similarity = synset.path_similarity(otro_synset)
            if similarity is not None:
                similarities.append((otro_synset, similarity))

        similarities.sort(key=lambda x: x[1], reverse=True)
        terminos_similares.extend((syn.name().split('.')[0] for syn, _ in similarities[:5]))

    return terminos_similares

# Función para obtener los términos más similares utilizando wup_similarity
def obtener_terminos_similares2(sustantivo):
    synsets = wn.synsets(sustantivo, pos=wn.NOUN)
    terminos_similares = []

    for synset in synsets:
        similarities = []
        for otro_synset in all_synsets_nouns:
            similarity = synset.wup_similarity(otro_synset)
            if similarity is not None:
                similarities.append((otro_synset, similarity))

        similarities.sort(key=lambda x: x[1], reverse=True)
        terminos_similares.extend((syn.name().split('.')[0] for syn, _ in similarities[:5]))

    return terminos_similares

############################################################################################################################################################

sustantivos_mas_comunes = []

#Identificar el sustantivo más común en cada capítulo
for archivo, etiquetas_oraciones in etiquetas_por_archivo.items():
    print(f'Archivo: {archivo}')
    sustantivos = []
    for oracion in etiquetas_oraciones:
        for palabra, etiqueta in oracion:
            # Filtrar sustantivos comunes y eliminar palabras vacías y palabras no deseadas
            if etiqueta.startswith('NN') and palabra not in stopwords.words('english') and palabra not in ['’', "'s", '“', '”', '*', 'Tommy', 'Alice']:
                sustantivos.append(palabra)
    # Calcular la frecuencia de los sustantivos
    fdist = FreqDist(sustantivos)
    sustantivo_mas_comun = fdist.most_common(1)
    
    # Guardar el sustantivo más común en la lista
    if sustantivo_mas_comun:
        sustantivos_mas_comunes.append(sustantivo_mas_comun[0][0])
    else:
        sustantivos_mas_comunes.append(None)
    
    # Imprimir el sustantivo más común encontrado en el archivo
    if sustantivo_mas_comun:
        print(f'Sustantivo más común encontrado: {sustantivo_mas_comun[0]}')
    else:
        print('No se encontraron sustantivos en este archivo.')
    
    print()

# Crear una lista para almacenar los términos similares para cada sustantivo más común
terminos_similares = []

# Iterar sobre cada sustantivo más común
for sustantivo in sustantivos_mas_comunes:
    if sustantivo is not None:  # Asegurarse de que el sustantivo no sea None
        terminos = obtener_terminos_similares(sustantivo)
        terminos_similares.append(terminos)
    else:
        terminos_similares.append([])

# Imprimir los términos similares para cada sustantivo más común
print(f'Terminos similares aplicando path_similarity')
for i, terminos in enumerate(terminos_similares):
    print(f'Términos similares para el sustantivo más común del archivo {capitulos[i]}:')
    print(terminos[0:5])
    print()


print(f'\n\n Terminos similares utilizando wup_similarity\n')
terminos_similares2 = []
# Iterar sobre cada sustantivo más común
for sustantivo in sustantivos_mas_comunes:
    if sustantivo is not None:  # Asegurarse de que el sustantivo no sea None
        terminos = obtener_terminos_similares2(sustantivo)
        terminos_similares2.append(terminos)
    else:
        terminos_similares2.append([])

for i, terminos in enumerate(terminos_similares2):
    print(f'Términos similares para el sustantivo más común del archivo {capitulos[i]}:')
    print(terminos[0:5])
    print()


Archivo: capitulo1libro1.txt
Sustantivo más común encontrado: ('Holmes', 23)

Archivo: capitulo1libro2.txt
Sustantivo más común encontrado: ('father', 18)

Archivo: capitulo1libro3.txt
Sustantivo más común encontrado: ('father', 13)

Archivo: capitulo1libro4.txt
Sustantivo más común encontrado: ('planet', 14)

Archivo: capitulo1libro5.txt
Sustantivo más común encontrado: ('Tuppence', 29)

Archivo: capitulo1libro6.txt
Sustantivo más común encontrado: ('John', 25)

Archivo: capitulo1libro7.txt
Sustantivo más común encontrado: ('part', 7)

Archivo: capitulo1libro8.txt
Sustantivo más común encontrado: ('way', 11)

Archivo: capitulo1libro9.txt
Sustantivo más común encontrado: ('driver', 26)

Archivo: capitulo1libro10.txt
Sustantivo más común encontrado: ('ghost', 34)

Terminos similares aplicando path_similarity
Términos similares para el sustantivo más común del archivo capitulo1libro1.txt:
['cognition', 'psychological_feature', 'abstraction', 'motivation', 'event']

Términos similares par

#Similitud de frases 

In [12]:
nlp = spacy.load("en_core_web_sm")

# Función para extraer frases clave usando TextRank
def top_words_spacy(text, max_phrase_length=10):
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Construir una lista de oraciones
    sentences = [sent.text for sent in doc.sents]
    
    # Calcular el puntaje de TextRank para cada oración
    wordsSpacy = []
    for sent in sentences:
        phrase_doc = nlp(sent)
        # Filtrar palabras vacías y signos de puntuación
        phrase_tokens = [token.text for token in phrase_doc if not token.is_stop and not token.is_punct]
        if len(phrase_tokens) <= max_phrase_length:
            wordsSpacy.append((sent, len(phrase_tokens)))
    
    # Ordenar las frases clave por puntaje de TextRank
    wordsSpacy = sorted(wordsSpacy, key=lambda x: x[1], reverse=True)
    return wordsSpacy[0:1]


guardar_frases_clave = []
# Extraer frases clave para cada capítulo
for archivo in capitulos:
    texto = new_corpus.raw(archivo)
    guardar_frases_clave.append(top_words_spacy(texto))

# Imprimir las frases clave extraídas
for i, frases_clave in enumerate(guardar_frases_clave):
    print(f'Frases clave para el archivo {capitulos[i]}:')
    for frase, _ in frases_clave:
        print(frase)
    print()

#Libro elegido Dracula para comparar su frase clave con la de los demás libros
frase_elegida = guardar_frases_clave[8]


def convertirTags(tag):
    # Convertir etiquetas POS de Penn Treebank a WordNet
    tag_dict = {'N': 'n', 'V': 'v', 'R': 'r', 'J': 'a'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

def convertirSynsets(frase):
    # Convertir una frase en una lista de synsets
    tokens = nltk.word_tokenize(frase)
    post = nltk.pos_tag(tokens)
    # Convertir etiquetas POS de Penn Treebank a WordNet
    tags = [tag[1] for tag in post]
    wntag = [convertirTags(tag) for tag in tags]
    ans = list(zip(tokens, wntag))
    # Obtener synsets
    sets = [wn.synsets(x,y) for x,y in ans]
    final = [val[0] for val in sets if len(val) > 0]
    # return synsets
    return final


def compararFrases(frase1, frase2):
    # Calcular la similitud de las frases clave
    s =[]
    for i1 in frase1:
        r = []
        # Comparar con cada synset de la otra frase
        scores = [x for x in [i1.path_similarity(i2) for i2 in frase2] if x is not None]
        if scores:
            s.append(max(scores))
    return sum(s)/len(s)

synset1 = convertirSynsets(str(frase_elegida[0]))
synset2 = convertirSynsets(str(guardar_frases_clave[0]))

for i in range(0, 10):
    synset2 = convertirSynsets(str(guardar_frases_clave[i]))
    print(f"Similitud entre las frases clave de Dracula y {capitulos[i]}:")
    print(compararFrases(synset1, synset2))
    print()


Frases clave para el archivo capitulo1libro1.txt:
They were admirable things for the observer—excellent for
drawing the veil from men’s motives and actions.

Frases clave para el archivo capitulo1libro2.txt:
He was induced
to look with extreme disapprobation, and at last with distaste, on my
father’s imprudence and follies.

Frases clave para el archivo capitulo1libro3.txt:
Beaufort had taken effectual measures to conceal himself, and it was ten
months before my father discovered his abode.

Frases clave para el archivo capitulo1libro4.txt:
The immediate pressure of necessity has brightened
their intellects, enlarged their powers, and hardened their hearts.

Frases clave para el archivo capitulo1libro5.txt:
The place was full, and they wandered about looking for a table,
catching odds and ends of conversation as they did so.



Frases clave para el archivo capitulo1libro6.txt:
Having no near relations or friends, I was trying to make
up my mind what to do, when I ran across John Cavend

#SIMILITUD SINTÁCTICA DE PALABRAS Y DOCUMENTOS

##Similitud de palabras con “embedding”

In [13]:
embeddings_diccionario = {}

#Cargar los vectores de palabras GloVe
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        palabra = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_diccionario[palabra] = vector
f.close()

# Función para encontrar palabras similares utilizando embeddings de palabras
def encontrar_similares(palabra, embeddings_diccionario, n=5):
    palabra = palabra.lower()
    # Verificar si la palabra está en el diccionario de embeddings
    if palabra in embeddings_diccionario:
        vector = embeddings_diccionario[palabra]
        similares = []
        for palabra, vector_palabra in embeddings_diccionario.items():
            # Calcular la similitud del coseno entre los vectores
            similitud = sp.spatial.distance.cosine(vector, vector_palabra)
            similares.append((palabra, similitud))
        # Ordenar por similitud y obtener las palabras más similares
        similares.sort(key=lambda x: x[1])
        return similares[1:n+1]
    else:
        return []
    
# Encontrar palabras similares para cada verbo más común
palabras_similares = []
print(verbos_mas_comunes)

for verbo in verbos_mas_comunes:
    palabras_similares.append(encontrar_similares(verbo, embeddings_diccionario))

# Imprimir las palabras similares para cada verbo más común
print(f'Palabras similares para cada verbo más común:\n')
for i, palabras in enumerate(palabras_similares):
    print(f'Palabras similares para el verbo más común del archivo {capitulos[i]}:')
    for palabra, similitud in palabras:
        print(f'{palabra} - Similitud: {1-similitud}')
    print()



['said', 'came', 'became', 'remember', 'know', 'said', 'go', 'see', 'seemed', 'said']
Palabras similares para cada verbo más común:

Palabras similares para el verbo más común del archivo capitulo1libro1.txt:
told - Similitud: 0.8487459435498669
says - Similitud: 0.8161521914923707
saying - Similitud: 0.7887590350807986
added - Similitud: 0.7826225953331531
asked - Similitud: 0.7763409947242162

Palabras similares para el verbo más común del archivo capitulo1libro2.txt:
took - Similitud: 0.8730760801920413
went - Similitud: 0.8710107969032627
last - Similitud: 0.8637165318123398
followed - Similitud: 0.8475635685297999
again - Similitud: 0.846613954267577

Palabras similares para el verbo más común del archivo capitulo1libro3.txt:
becoming - Similitud: 0.8460562553656712
become - Similitud: 0.8415305786329104
was - Similitud: 0.8205906975345288
as - Similitud: 0.7907937057505346
later - Similitud: 0.7635584429852055

Palabras similares para el verbo más común del archivo capitulo1libro

##Similitud de documentos con “embedding”

In [14]:
# Cargar el modelo y el tokenizer de BERT
model = transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

model.eval()

text1 = str(frase_elegida[0])

# Iterar sobre cada frase clave y calcular la similitud con el primer capítulo de Dracula
for i in range(len(guardar_frases_clave)): 
    # Codificar las frases clave
    tokens1 = tokenizer.encode(text1, return_tensors='pt', truncation=True, padding=True)
    tokens2 = tokenizer.encode(str(guardar_frases_clave[i][0]), return_tensors='pt', truncation=True, padding=True)

    # Obtener embeddings
    with torch.no_grad():
        outputs1 = model(input_ids=tokens1)
        outputs2 = model(input_ids=tokens2)
    
    # Obtener el embedding de la primera posición
    embedding1 = outputs1.last_hidden_state[:, 0, :].numpy()
    embedding2 = outputs2.last_hidden_state[:, 0, :].numpy()

    # Calcular la similitud del coseno entre los embeddings
    similarity = np.dot(embedding1.flatten(), embedding2.flatten()) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    print(f"Similitud entre las frases clave de Dracula y {capitulos[i]}:  {similarity}")
    print(text1)
    print(str(guardar_frases_clave[i][0]))
    print()




c:\Users\davic\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Similitud entre las frases clave de Dracula y capitulo1libro1.txt:  0.9180206060409546
('I had for dinner, or\r\nrather supper, a chicken done up some way with red pepper, which was\r\nvery good but thirsty.', 10)
('They were admirable things for the observer—excellent for\r\ndrawing the veil from men’s motives and actions.', 10)

Similitud entre las frases clave de Dracula y capitulo1libro2.txt:  0.9333732724189758
('I had for dinner, or\r\nrather supper, a chicken done up some way with red pepper, which was\r\nvery good but thirsty.', 10)
('He was induced\r\nto look with extreme disapprobation, and at last with distaste, on my\r\nfather’s imprudence and follies.', 10)

Similitud entre las frases clave de Dracula y capitulo1libro3.txt:  0.9260679483413696
('I had for dinner, or\r\nrather supper, a chicken done up some way with red pepper, which was\r\nvery good but thirsty.', 10)
('Beaufort had taken effectual measures to conceal himself, and it was ten\r\nmonths before my father disc